# Mock Trading System - Example Notebook

This notebook demonstrates how to use the mock trading system components.

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

## 1. Fetching Stock Data

In [ ]:
from data_fetcher import DataFetcher, get_sample_tickers

# Initialize fetcher with 180 days lookback
fetcher = DataFetcher(lookback_days=180)

# Fetch single stock
aapl = fetcher.fetch_stock_data('AAPL')
print(f"AAPL data shape: {aapl.shape}")
aapl.tail()

In [ ]:
# Plot price and volume
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=aapl.index,
    open=aapl['Open'],
    high=aapl['High'],
    low=aapl['Low'],
    close=aapl['Close'],
    name='AAPL'
))
fig.update_layout(title='AAPL Price', xaxis_rangeslider_visible=False)
fig.show()

## 2. Generating Forecasts with Chronos-2

In [ ]:
from forecaster import ChronosForecaster

# Initialize forecaster (will use fallback if Chronos not installed)
forecaster = ChronosForecaster(
    model_size='bolt-small',
    prediction_length=5
)

# Generate forecast
forecast = forecaster.forecast(aapl['Close'], 'AAPL')

print(f"Expected Return: {forecast.expected_return:.2%}")
print(f"Forecast Volatility: {forecast.forecast_volatility:.2%}")
print(f"Confidence Score: {forecast.confidence_score:.2f}")

In [ ]:
# Plot forecast with confidence interval
fig = go.Figure()

# Historical
fig.add_trace(go.Scatter(
    x=aapl.index[-30:],
    y=aapl['Close'].iloc[-30:],
    mode='lines',
    name='Historical'
))

# Forecast
fig.add_trace(go.Scatter(
    x=forecast.forecast_dates,
    y=forecast.point_forecast,
    mode='lines+markers',
    name='Forecast',
    line=dict(dash='dash')
))

# Confidence interval
fig.add_trace(go.Scatter(
    x=list(forecast.forecast_dates) + list(forecast.forecast_dates[::-1]),
    y=list(forecast.upper_bound) + list(forecast.lower_bound[::-1]),
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    name='80% CI'
))

fig.update_layout(title='AAPL Price Forecast')
fig.show()

## 3. Generating Trading Signals

In [ ]:
from strategy import TradingStrategy, Signal

# Initialize strategy
strategy = TradingStrategy(
    return_threshold_buy=0.02,
    return_threshold_sell=-0.02,
    max_position_weight=0.25
)

# Generate signal
signal = strategy.generate_signal(
    forecast=forecast,
    portfolio_value=100000,
    current_position=0
)

print(f"Signal: {signal.signal.value}")
print(f"Target Weight: {signal.target_weight:.2%}")
print(f"Position Size: ${signal.position_size:,.2f}")
print(f"Shares: {signal.shares}")
print(f"Stop Loss: ${signal.stop_loss:.2f}")
print(f"Take Profit: ${signal.take_profit:.2f}")
print(f"Rationale: {signal.rationale}")

## 4. Portfolio Management

In [ ]:
from portfolio import Portfolio

# Initialize portfolio
portfolio = Portfolio(initial_capital=100000)

# Execute the trade
current_price = aapl['Close'].iloc[-1]
trade = portfolio.execute_trade(signal, current_price=current_price)

if trade:
    print(f"Executed: {trade.side} {trade.shares} shares @ ${trade.price:.2f}")

print(f"\nPortfolio Summary:")
print(f"  Cash: ${portfolio.cash:,.2f}")
print(f"  Positions Value: ${portfolio.positions_value:,.2f}")
print(f"  Total Value: ${portfolio.total_value:,.2f}")

In [ ]:
# View positions
portfolio.get_positions_summary()

## 5. Running a Backtest

In [ ]:
from backtester import Backtester, BacktestConfig, generate_backtest_report
from datetime import datetime, timedelta

# Configure backtest
config = BacktestConfig(
    start_date='2024-01-01',
    end_date='2024-06-30',
    tickers=['AAPL', 'MSFT', 'GOOGL'],
    initial_capital=100000,
    rebalance_frequency='weekly',
    forecast_horizon=5
)

print(f"Backtest Configuration:")
print(f"  Period: {config.start_date} to {config.end_date}")
print(f"  Tickers: {config.tickers}")
print(f"  Capital: ${config.initial_capital:,.2f}")

In [ ]:
# Run backtest (this may take a few minutes)
backtester = Backtester()

try:
    result = backtester.run_backtest(config)
    print(generate_backtest_report(result))
except Exception as e:
    print(f"Backtest error: {e}")
    print("Note: Full backtest requires market data access.")

## 6. Visualizing Results

In [ ]:
# If backtest was successful, plot equity curve
if 'result' in dir() and result is not None:
    equity_data = pd.DataFrame([
        {'Date': s.timestamp, 'Value': s.total_value}
        for s in result.portfolio_history
    ])
    
    fig = px.line(equity_data, x='Date', y='Value', title='Portfolio Equity Curve')
    fig.add_hline(y=config.initial_capital, line_dash='dash', line_color='gray')
    fig.show()